In [44]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample


In [36]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [ ]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

balanceamento dos dados

In [55]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()

In [56]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64


In [57]:
from sklearn.utils import resample

# Separar cada classe
class_min = train[train['classe'] == -1]
class_med = train[train['classe'] == 1]
class_max = train[train['classe'] == 0]

# Número da classe maior
n_samples = len(class_max)

# Replicar as classes menores até o tamanho da maior
class_min_upsampled = resample(class_min, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

class_med_upsampled = resample(class_med, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

train = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

# baralhar o conjunto de treino
train = train.sample(frac=1, random_state=42).reset_index(drop=True)

print(train['classe'].value_counts())


classe
 1    2375
 0    2375
-1    2375
Name: count, dtype: int64


tokeinização

In [59]:

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
print(train['stems'].head())

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [a, qued, ocorr, aliment, pux, cord, sent, con...
1    [a, repercuss, nega, cas, fez, bolsonar, ,, ca...
2    [o, brasil, lav, alm, após, decepcion, empat, ...
3    [ao, final, encontr, ,, sen, tucan, diss, esp,...
4    [fiz, compromiss, líd, bas, hoj, líd, opos, ,,...
Name: stems, dtype: object


Análise exploratória

# Construção dos modelos de representação

## TF-IDF - fred

In [ ]:
def TF_IDF():
    tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
    max_df =0.5 , max_features =500)
    p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [60]:
def count_vec(train,val,test):
    c_vect = CountVectorizer(ngram_range=(1,1), strip_accents='unicode',max_features=1000)

    #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))
    
    #print("Text train:\n", text_train.head())

    # Transformar em vetores
    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)
    X_test  = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [61]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train,val,test)

## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-cbow-300d

In [62]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file
from gensim.models import KeyedVectors
def build_word2vec_model():
    #descarregar os embeddings do modelo Word2Vec CBOW 300d do NILC
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                           filename="embeddings.safetensors")   
    
    #carregar os vetores do ficheiro safetensors
    data = load_file(path)
    vectors = data["embeddings"]    

    #descarregar vocabulário
    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                                 filename="vocab.txt")
    
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]  

    #print(vectors.shape)
    #print(vectors)

    model = KeyedVectors(vector_size=300)
    model.add_vectors(vocab, vectors)

    print(f"Número de palavras no modelo: {len(model.key_to_index)}")
    print(f"Dimensão dos vetores: {model.vector_size}")

In [ ]:
'''from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)'''

# Aplicar modelos de classificação

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def Naive_Bayes(X_train, y_train, X_val, y_val, X_test, alphas):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 


Count Vectorizer

In [77]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Melhor alpha encontrado: 6.0 (F1 val = 0.7262)
              precision    recall  f1-score   support

           1       0.56      0.65      0.60       417
           0       0.84      0.68      0.75      1273
          -1       0.20      0.40      0.27       168

    accuracy                           0.65      1858
   macro avg       0.53      0.58      0.54      1858
weighted avg       0.72      0.65      0.67      1858

[[269 102  46]
 [179 869 225]
 [ 33  67  68]]


## SVM

In [66]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [67]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

[I 2025-11-09 11:31:50,213] A new study created in memory with name: no-name-c223fbf7-a894-4172-a36e-fc39ff3efbbb


----COUNT VECTORIZER ----


[I 2025-11-09 11:31:58,101] Trial 6 finished with value: 0.5653089344462279 and parameters: {'C': 7.610893690323544, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 6 with value: 0.5653089344462279.
[I 2025-11-09 11:31:58,274] Trial 3 finished with value: 0.5789194953944149 and parameters: {'C': 0.3803183291697106, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 3 with value: 0.5789194953944149.
[I 2025-11-09 11:31:59,955] Trial 1 finished with value: 0.5721791182220497 and parameters: {'C': 0.2121674689877027, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 3 with value: 0.5789194953944149.
[I 2025-11-09 11:32:02,298] Trial 5 finished with value: 0.5958994211584275 and parameters: {'C': 0.0531697438099838, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 5 with value: 0.5958994211584275.
[I 2025-11-09 11:32:04,031] Trial 0 finished with value: 0.5938006537380964 and parameters: {'C': 0.6640210956784931, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 5 with value: 0.59

Melhores hiperparâmetros encontrados:
{'C': 0.011154814601379051, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

           1       0.50      0.66      0.57       417
           0       0.84      0.70      0.76      1273
          -1       0.27      0.40      0.33       168

    accuracy                           0.66      1858
   macro avg       0.54      0.59      0.55      1858
weighted avg       0.71      0.66      0.68      1858

[[277 106  34]
 [238 887 148]
 [ 34  66  68]]


## KNN

In [68]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [69]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Best k: 1
              precision    recall  f1-score   support

           1       0.28      0.68      0.40       417
           0       0.78      0.46      0.58      1273
          -1       0.12      0.08      0.10       168

    accuracy                           0.47      1858
   macro avg       0.39      0.41      0.36      1858
weighted avg       0.61      0.47      0.49      1858

[[282 110  25]
 [615 582  76]
 [ 96  58  14]]


## Decison Tree

In [70]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [71]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Melhor alpha: 0.0005 
              precision    recall  f1-score   support

           1       0.49      0.41      0.45       417
           0       0.75      0.83      0.79      1273
          -1       0.25      0.15      0.19       168

    accuracy                           0.68      1858
   macro avg       0.50      0.46      0.48      1858
weighted avg       0.65      0.68      0.66      1858

[[ 173  230   14]
 [ 153 1057   63]
 [  25  118   25]]
